In [1]:
import numpy as np
import json

try:
    with open("joint_data.json") as file:
        joint_data = json.load(file)
except FileNotFoundError:
    joint_data = {}
    print('No such file on disk')

No such file on disk


In [2]:
'''
TODO: Append 2D-vectors (tuples) together
      Make appropriate assertions
'''

joint_positions = {}
for root_key in joint_data:
    nested_dict = joint_data[root_key]
    for joint_key in nested_dict:
        joint_position = np.asarray(nested_dict[joint_key]).reshape(2,)
        
        if not joint_key in joint_positions:
            joint_positions[joint_key] = joint_position
        else:
            prev = joint_positions[joint_key]
            joint_positions[joint_key] = np.append(prev, joint_position, axis=0)

In [6]:
def avgfilter (inp, npoints):
    
    '''
    TODO: Make workable with 2xN array
          Make appropriate assertions
    '''
    
    coef = 1. / float(npoints)
    vals = np.concatenate((np.zeros((npoints,)), inp), axis=0)    
    end = len(vals)
    
    y = np.zeros_like(inp)
    for p in range(npoints):
        y = y + vals[npoints-p:end-p]*coef
    
    return y

def assert_filt(c1, c2):
    assert len(c1) == len(c2), \
        'leny (%d) != lenp (%d)' %(len(c1), len(c2))

    
joint_velocities = {}
joint_accelerations = {}
for joint in joint_positions:
    positions = joint_positions[joint]
    y = avgfilter(positions, npoints=2)
    assert_filt(y, positions)
    joint_positions[joint] = y
    
    '''
    TODO: Compute euclidean distances at accel/veloc estimations
    '''
    
    # assume velocity of first reading is zero
    velocities = np.zeros_like(positions)
    velocities[1:] = positions[1:] - positions[0:len(positions)-1] # !!! Euclidean distance
    y = avgfilter(velocities, npoints=7)
    assert_filt(y, velocities)
    joint_velocities[joint] = y
    
    # assume acceleration of first reading is zero
    accelerations = np.zeros_like(positions)
    accelerations[1:] = velocities[1:] - velocities[0:len(velocities)-1] # !!! Euclidean distance
    y = avgfilter(accelerations, npoints=7)
    assert_filt(y, accelerations)
    joint_accelerations[joint] = y
    
    

In [26]:
pp = np.arange(36).reshape((18, 2))

In [28]:
pp.shape
print(pp[0:5])




[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
